# Path

In [150]:
import sys
import os

project_path = r'/home/craxiss/Documents/projects/kapsul_etkinlik_v2'

os.chdir(project_path)
sys.path.append(project_path)

# Import

In [151]:
from helpers import column_formatter, utf8_convert, first_letter_id_generator
from charset_normalizer import detect
from config import data_path as path
from string import ascii_lowercase
from itertools import combinations 
from json import load, dump
import pandas as pd
import warnings
import openpyxl

# Read

In [152]:
information = pd.read_excel(
    path+'/3-result/information.xlsx', index_col='Unnamed: 0')
events_per_person = pd.read_excel(
    path+'/3-result/events_per_person.xlsx', index_col='Unnamed: 0')
events = pd.read_excel(
    path+'/3-result/events.xlsx', index_col='Unnamed: 0')
ohe = pd.read_excel(
    path+'/3-result/event_per_person_ohe.xlsx', index_col='Unnamed: 0')


with pd.ExcelFile(path + '/2-cleaned/basvuru.xlsx') as appeals_xlsx:
    appeals = pd.read_excel(
        appeals_xlsx, index_col='Unnamed: 0', sheet_name='Başvurular')
    lessons_all = pd.read_excel(
        appeals_xlsx, index_col='Unnamed: 0', sheet_name='Dersler')


with pd.ExcelFile(path + '/2-cleaned/yoklama.xlsx') as rolls_xlsx:
    rolls_info = pd.read_excel(
        rolls_xlsx, 'General Info', index_col='Unnamed: 0')

    rolls = dict()
    for sheet in rolls_info.index:
        rolls[sheet] = pd.read_excel(rolls_xlsx, sheet)

information['id'] = information.index

# Modify Event.xlsx

## Get Appeal & Roll Count

In [153]:
appeals_list, rolls_list = list(), list()
for i in range(1,25):
    appeals_list.append(ohe[f'b{i}'].sum())
    rolls_list.append(ohe[f'k{i}'].sum())

events['basvuru'], events['katılım'] = appeals_list, rolls_list

events['oran'] = (events['katılım'] / events['basvuru'])* 100

In [154]:
groups = ['ay', 'gün', 'saat', 'süre']

comb = dict()


def merger(df1, df2, df3, df4, df5=None):
    df1['basvuru'] = df4['basvuru']
    df1['etkinlik sayısı'] = df2['eğitim']
    df1['etkinlikler'] = df3['eğitim']
    # df1['egitimciler'] = df5['eğitimci']
    return df1


with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    # all
    comb[" ".join(groups)] = merger(
        events.groupby(by=groups).sum()[['katılım']],
        events.groupby(by=groups).count()[['eğitim']],
        pd.DataFrame(events.groupby(by=groups)[['eğitim']].sum()[
                     'eğitim'].str.split(' 101').str.join(', ').str[:-2]),
        events.groupby(by=groups).sum()[['basvuru']]
    )

    # 3 combination
    for iter in list(combinations(groups, 3)):
        comb[" ".join(iter)] = merger(
            events.groupby(by=list(iter)).sum()[['katılım']],
            events.groupby(by=list(iter)).count()[['eğitim']],
            pd.DataFrame(events.groupby(by=list(iter))[['eğitim']].sum()[
                         'eğitim'].str.split(' 101').str.join(', ').str[:-2]),
            events.groupby(by=list(iter)).sum()[['basvuru']]
            )

    # 2 combination
    for iter in list(combinations(groups, 2)):
        comb[" ".join(iter)] = merger(
            events.groupby(by=list(iter)).sum()[['katılım']],
            events.groupby(by=list(iter)).count()[['eğitim']],
            pd.DataFrame(events.groupby(by=list(iter))[['eğitim']].sum()[
                         'eğitim'].str.split(' 101').str.join(', ').str[:-2]),
            events.groupby(by=list(iter)).sum()[['basvuru']]
            )

    # one
    for group in groups:
        comb[group] = merger(
            events.groupby(by=group).sum()[['katılım']],
            events.groupby(by=group).count()[['eğitim']],
            pd.DataFrame(events.groupby(by=group)[['eğitim']].sum()[
                         'eğitim'].str.split(' 101').str.join(', ').str[:-2]),
            events.groupby(by=group).sum()[['basvuru']]
            )


# Write

In [155]:
with pd.ExcelWriter(path+'3-result/groupby.xlsx', mode='w') as writer:
    for group,df in comb.items():
        df.to_excel(writer, sheet_name=group)


book = openpyxl.load_workbook(path+'3-result/events.xlsx')
with pd.ExcelWriter(path+'3-result/events.xlsx', engine='openpyxl') as writer:
    writer.book = book
    events.to_excel(writer, sheet_name='Events-Modified')